### This notebook calculates parameters for ALMA input and interpret result from ALMA output
This notebook has to be run with Python 3.8.10 kernel

In [13]:
# Anyways I can do some caluclations of parameters first or set up a basic script

# Required Resolution
# prob about 0.1 arcsec and maybe smaller
#To calculate

In [14]:
# import casatasks
#https://github.com/aardk/jupyter-casa/blob/master/examples/vla-cont-tutorial.ipynb
#https://colab.research.google.com/github/casangi/examples/blob/master/community/casa6_demo.ipynb#scrollTo=IIr5sp0VgR2m

### There is a sensitivity calculator already

https://almascience.nrao.edu/proposing/sensitivity-calculator


Common Parameters:


Declination:  14h08m10s

Polarisation:  Dual



### Frequency to Wavelength Conversion
| Band | 1 | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  |
|----------------|-----|------|------|------|------|------|------|------|------|
| Frequency (GHz) | 40  | 100  | 150  | 185  | 230  | 345  | 460  | 650  | 870  |
| Wavelength (μm) | 7500 | 3000 | 2000 | 1621.62 | 1304.35 | 869.57 | 652.17 | 461.54 | 344.83 |

### Spatial Resolution Formula in Arcseconds

The resolution in arcseconds can be approximated as:

$$
\text{FWHM} (") = \frac{76}{\text{max\_baseline (km)} \times \text{frequency (GHz)}}
$$


### Maximum Recoverable Scale (MRS):

$$
\text{MRS} \approx 0.6 \times \frac{\lambda}{b_{\text{min}}}
$$


<p style="color:blue;">This is a note for the maximum baseline.</p>

In [2]:
# Take the radius of a CPD as 1 au (roughly for a HD168142b by calculating Hll's radius))
# Rmb AU is not parsec-dependent, just stands for Earth-Sun distance

import numpy as np
import pandas as pd
from astropy import units as u
from astropy.constants import au, pc
from tabulate import tabulate

# Convert constants to CGS values
au = au.cgs.value  # Astronomical Unit in cm
pc = pc.cgs.value  # Parsec in cm

# Define the radius of the CPD and the distance to the object
radius_cpd = 1 * au  # 1 AU in cm
distance_to_cpd = 100 * pc  # 100 parsecs in cm

# Calculate the angular size in radians (small-angle approximation)
angular_size_rad = radius_cpd / distance_to_cpd  # θ ≈ r / d for small angles

# Convert radians to arcseconds
angular_size_arcsec = (angular_size_rad * u.rad).to(u.arcsec)

print(f"The angular size of the CPD is {angular_size_arcsec:.3f}, or {angular_size_arcsec.to(u.mas):.3f} in mas.")

# Function to create ALMA table with configurations
def create_alma_table():
    """Create a Pandas DataFrame for ALMA configurations C-7 to C-10 with band row"""
    
    # Define columns
    columns = ["Config", "b_max (km)", "40 GHz", "100 GHz", "150 GHz", "185 GHz", 
               "230 GHz", "345 GHz", "460 GHz", "650 GHz", "870 GHz"]
    
    # Data excluding the "Band" row for consistency
    data = [
        ["C-7", 3.6, 0.53, 0.21, 0.14, 0.11, 0.092, 0.061, 0.046, 0.033, 0.024],
        ["C-8", 8.5, 0.24, 0.096, 0.064, 0.052, 0.042, 0.028, 0.021, 0.015, 0.011],
        ["C-9", 13.9, 0.14, 0.057, 0.038, 0.031, 0.025, 0.017, 0.012, 0.0088, 0.0066],
        ["C-10", 16.2, 0.11, 0.042, 0.028, 0.023, 0.018, 0.012, 0.0091, 0.0065, 0.0048]
    ]
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

# Create the ALMA table
df = create_alma_table()

# Store angular resolution values in a universal NumPy array
angular_resolutions = df.iloc[:, 2:].to_numpy()  # Extract only frequency-dependent columns


# Extract individual arrays
C7_res = angular_resolutions[0]  # C-7 values
C8_res = angular_resolutions[1]  # C-8 values
C9_res = angular_resolutions[2]  # C-9 values
C10_res = angular_resolutions[3]  # C-10 values


# Print the DataFrame using tabulate for better formatting
print(tabulate(df, headers='keys', tablefmt='plain'))

#print(tabulate(df, headers='keys', tablefmt='latex'))

# Observational frequency and ALMA Band reference
obs_frequency = 300 * u.GHz  # Observing at 1000 nm
print(f'\nBand2 not available for cycle 11?')
print(f'\nIf I observe at 1000 nm, the frequency is {obs_frequency} (Band 7)')
print(f'Ignoring the maximum recoverable scale as no extended emission observation is being performed.')


The angular size of the CPD is 0.010 arcsec, or 10.000 mas in mas.
    Config      b_max (km)    40 GHz    100 GHz    150 GHz    185 GHz    230 GHz    345 GHz    460 GHz    650 GHz    870 GHz
 0  C-7                3.6      0.53      0.21       0.14       0.11       0.092      0.061     0.046      0.033      0.024
 1  C-8                8.5      0.24      0.096      0.064      0.052      0.042      0.028     0.021      0.015      0.011
 2  C-9               13.9      0.14      0.057      0.038      0.031      0.025      0.017     0.012      0.0088     0.0066
 3  C-10              16.2      0.11      0.042      0.028      0.023      0.018      0.012     0.0091     0.0065     0.0048

Band2 not available for cycle 11?

If I observe at 1000 nm, the frequency is 300.0 GHz (Band 7)
Ignoring the maximum recoverable scale as no extended emission observation is being performed.


### Noise Level in an Interferometer

The noise level in the resulting data cubes (expressed in mJy) for an interferometer scales as:

$$
\sigma = \frac{k T_{\text{sys}}}{A N^2 \sqrt{N_p \Delta\nu \Delta t}}
$$

where:


$$
\begin{aligned}
\sigma &= \text{Noise level (mJy)} \\
k &= \text{Boltzmann’s constant} \\
T_{\text{sys}} &= \text{System temperature} \\
A &= \text{Area of each antenna} \\
N &= \text{Number of antennas} \\
N_p &= \text{Number of polarizations} \\
\Delta\nu &= \text{Available bandwidth} \\
\Delta t &= \text{Observing time}
\end{aligned}
$$


$$ S_{\text{Jy/beam}} =  S_{\text{Jy/pixel}} \times \frac{\Omega_{\text{beam}}}{\Omega_{\text{pixel}}} $$


### Surface Brightness Sensitivity Formula

The surface brightness sensitivity for non-point source is related to the point-source sensitivity  by:

$$ \sigma_T = \frac{\sigma_S \lambda^2}{2k \Omega} $$

where the beam solid angle is given by:

$$ \Omega = \frac{\pi \theta_{res}^2}{4 \ln 2} $$

where theta is the spatial resolution (FWHM of the beam).



In [17]:
# I want to estimate the noise I need to detect the CPD with a 5 sigma confidence
# Then I can use the ALMA Sensitivity Calculator to estimate the integration time

# Radmc3d(Jy/pixel) to (ALMA)Jy/beam and integrate over the beam and divide the total flux by 5
# The beam is just the angular resolution of the band
# RADMC3d  pixel size is 0.1 arcsec
#  pixel size in arcsec = ( pixel size in cm / 1.496e13) / (distance in parsec)
# pixel size in cm can be found in image.out (4th row)

#######################################################################################################################################################################
# Relevant Pixel/Beam/Arcsecond conversion



import radmc3dPy.image as image

# Read the image.out file
img = image.readImage('image.out')

# Get the pixel size
nx = img.nx
ny = img.ny
pixel_size_x_cm = img.sizepix_x  #cm
pixel_size_y_cm = img.sizepix_y #cm

dpc = 100 #distance_to_object_pc

# Print the pixel size
print(f"The image size is {nx} pixels wide and {ny} pixels high.")
print(f"The pixel size is {pixel_size_x_cm} cm in x-direction and {pixel_size_y_cm} cm in y-direction.")


# Convert the pixel size to arcseconds
pixel_size_x_arcsec = ((pixel_size_x_cm / 1.496e13)/ dpc)  #arcsecond
pixel_size_y_arcsec = ((pixel_size_y_cm  / 1.496e13)/ dpc) #arcsecond

print(f"The pixel size is {pixel_size_x_arcsec:0.3f} arcseconds in x-direction and {pixel_size_y_arcsec:0.3f} arcseconds in y-direction.")

# Calculate the beam

beam_radmc = pixel_size_x_arcsec*pixel_size_y_arcsec # radmc3d beam in arcsec^2 
# or just the square area???

# Each band+config has a different beam size
# Need a matrix, how do I represent them so they are convenient when I need a noise level and not make it too messy?
# Start with just 1 configuration -C-10 


beam_ALMA_C10 = np.pi*(C10_res**2)/(4*np.log(2)) # for each band in C10 configuration


bands = [1, 3, 4, 5, 6, 7, 8, 9, 10]  # Correct band numbers


pixels_per_beam = [beam / beam_radmc for beam in beam_ALMA_C10]

# Print the beams in the desired format
print('\n')
print(f"The beam for radmc3d is {beam_radmc:.3e} arcsec^2")
print('\n')
print("For C10 configuration:")
print("Band\tBeam (arcsec^2)\tPixels per Beam")
for band, beam, pixels in zip(bands, beam_ALMA_C10, pixels_per_beam):
    print(f"Band {band}\t\t{beam:.3e}\t\t{pixels:.3f}")

print('\n')

print('For larger Bands, the pixel size is larger than the beam size')

Reading image.out
The image size is 100 pixels wide and 100 pixels high.
The pixel size is 29620382642004.49 cm in x-direction and 29620382642004.49 cm in y-direction.
The pixel size is 0.020 arcseconds in x-direction and 0.020 arcseconds in y-direction.


The beam for radmc3d is 3.920e-04 arcsec^2


For C10 configuration:
Band	Beam (arcsec^2)	Pixels per Beam
Band 1		1.371e-02		34.973
Band 3		1.999e-03		5.099
Band 4		8.883e-04		2.266
Band 5		5.994e-04		1.529
Band 6		3.671e-04		0.936
Band 7		1.632e-04		0.416
Band 8		9.383e-05		0.239
Band 9		4.787e-05		0.122
Band 10		2.611e-05		0.067




In [19]:
# Get the brightness of CPD outer rim in Jy/pixel * beam_ALMA/beam_radmc

ALMA1621_Jy_pixel = 8.769799660513e-5 # mean brightness over 4 pixels Jy/pixel   

noise = (ALMA1621_Jy_pixel/5)*beam_ALMA_C10[3]/beam_radmc

print(f'The noise level required to detect the CPD with a 5 sigma confidence is {noise:.3e} Jy/beam in Band 5 in config 10')
print(f'6.68853 hours is required')

The noise level required to detect the CPD with a 5 sigma confidence is 2.682e-05 Jy/beam in Band 5 in config 10
6.68853 hours is required


### CASA Simobserve parameters
